In [1]:
import tANS_py.tANS, tANS_py.Coder
import numpy as np

In [2]:
def normally_distributed_random_ints(mean, std_dev, size, low=None, high=None, dtype=int):
    # Generate normally distributed random floats
    normal_floats = np.random.normal(loc=mean, scale=std_dev, size=size)
    
    # Round floats to nearest integers
    rounded_ints = np.round(normal_floats).astype(dtype)
    
    if low is not None and high is not None:
        # Clip the values to be within the specified range
        rounded_ints = np.clip(rounded_ints, low, high)
    
    return rounded_ints

In [3]:
msg = [1,2,43,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

msg = np.random.randint(0, 256, 1020, dtype=np.uint8)

msg = normally_distributed_random_ints(128, 64, 1020, 0, 255, dtype = np.uint8)

# Test
res = tANS_py.tANS.encode_decode_test(msg, dtype = 'np')
res[2], msg.nbytes, len(res[0]) / 8, np.array_equal(msg, res[1])

NUMPYY
1020


(1.0458856703409383, 1020, 975.25, True)

In [4]:
import tANS_py.Utils as Utils
s = list(set(msg))
sum(Utils.rescale_list_to_power_of_2([np.sum(msg == i) for i in s], 2**10))
sum((np.sum(msg == i) for i in s))

1020

In [5]:
2**15

32768

In [16]:
import bisect

class CompPoint:
    def __init__(self, point, s_tab, dtype = "u"):
        # Initialize CompPoint with given point and symbol table (s_tab)
        self.dtype = dtype
        self.s_tab = s_tab

        self.point = point
        self.symbol = self.get_symbol()
        self.off = self.get_off()
        self.OL = self.s_tab["OL"][self.symbol]
        
    def __str__(self):
        # String representation of CompPoint
        return "Point: %d, Symbol: %d, Offset: %d" % (self.point, self.symbol, self.off)
        
    def get_symbol(self):
        # Use binary search to find the symbol
        vmin = self.s_tab["vmin"]
        if self.dtype == "s":
            temp_point = self.point + 128
            idx = bisect.bisect_right(vmin, temp_point) - 1
        else:
            idx = bisect.bisect_right(vmin, self.point) - 1
        
        # check if the point is in the range of the symbol
        # if self.point == vmin[idx]:
        #     idx += 1
            
        return idx#idx - 1 > 0 and idx - 1 or 0
            
    def get_off(self):
        # Calculate the offset from the symbol
        if self.dtype == "s":
            return self.point - self.s_tab["vmin"][self.symbol] + 128
        else:
            return self.point - self.s_tab["vmin"][self.symbol]

class CompTensor:
    def __init__(self, tensor, s_tab):
        # Initialize CompTensor with given tensor and symbol table (s_tab)
        self.tensor = tensor
        self.shape = tensor.shape
        self.flat = tensor.flatten()
        
        # Get the symbols and offsets for each point (as a list of CompPoints)
        self.points = [CompPoint(point, s_tab) for point in self.flat]

    def update_tensor(self):
        # Update tensor with new values from self.flatten
        self.tensor = self.flatten.reshape(self.shape)
        
    def update_flatten(self):
        # Update flatten with new values from self.tensor
        self.flat = self.tensor.flatten()
        
    def to(self, type):
        # Convert tensor to a specified type and update flatten
        self.tensor = self.tensor.to(type)
        self.update_flatten()
        
        return self

def int_to_binary_list(value, nbits):
        if value >= 2**nbits or value < 0:
            raise ValueError(f"Value {value} cannot be represented in {nbits} bits.")
        
        binary_list = [int(bit) for bit in bin(value)[2:].zfill(nbits)]
        return binary_list

In [44]:
import pandas as pd
n = "weight_13_flat"
msg = np.load(f"/Users/adamtaback/Downloads/{n}.npy")
print(msg.shape)
# test
res = tANS_py.tANS.encode_decode_test(msg, L= 2**8)
print("Comp Ratio: {}, Orig bytes: {}, Comp bytes: {}, Working: {}".format(res[2], msg.nbytes, len(res[0]) / 8, np.array_equal(msg, res[1])))

# Use symbol, offset
s_tab = pd.read_csv(f"/Users/adamtaback/Downloads/{n}.apack", sep = " ", header = None)
s_tab.columns = ["vmin","OL","abits","obits","vcnt"]
comp_tensor = [CompTensor(msg, s_tab)]

freq = list(s_tab["vcnt"])
freq = Utils.rescale_list_to_power_of_2(freq, 2**8)
offsets = []
for i in range(len(comp_tensor[0].points)):
    offsets.extend(int_to_binary_list(comp_tensor[0].points[i].off, comp_tensor[0].points[i].OL))
    
c = tANS_py.Coder.Coder(sum(freq), [i for i in range(len(freq))], freq)

msg_temp = [p.symbol for p in comp_tensor[0].points]
bits = c.encode(msg_temp)
1/((len(bits) + len(offsets)) / 8 / msg.nbytes)

(1728,)
NUMPYY
1728
Comp Ratio: 1.304643261608154, Orig bytes: 1728, Comp bytes: 1324.5, Working: True


1.3016949152542374

In [48]:
list("ALS")

['A', 'L', 'S']

In [47]:
len(offsets)

6912

In [24]:
comp_tensor[0]

In [46]:
alpha = [str(i) for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]]

# create a message with all the characters in the alphabet, randomly shuffled
import random

msg2 = ''.join(random.sample(alpha, len(alpha)))

# Test
res = tANS_py.tANS.encode_decode_test(msg2, dtype = 'str')
res[2], tANS_py.Coder.Coder.calculate_bits(msg2), len(res[0]) / 8, msg2 == res[1]

(2.164705882352941, 23, 10.625, True)

In [25]:
msg2

'ŭņïŁŞ²ĭƃü8¡ňÿĘƔzƇhƪtxĎƒÍŶĩē·Ŝ¸łƯĖċđ,ÐƆƝ®ŉ]èĄ{ƀƂ-4ƣƍâ¤ÅnÚ¿îÉŨŮôİĹ=ƠƓƉĈÛĐƶÑ3ĥŰÀL}ķǂØƹĻŴƈàŬNŐę)ŪµÔƵ?½ĒĪðfưSÞ¼PÆš©ƊFŔůœ`ğŤ×ŒƗsĀ¶őźĸã¾ĲƸc;ƨĿ@ùƦ§È÷¨\'#ŵ.ÝßÇm±ĆƷ¯ġĮ1kGožſ"ĤĦÕĠÄA«ŕ5Ō*JÏ[7/2ª(+uƟǀÂ$>ĜYĶ%ĕ~Ə<ĝCŏųŝ\xadýļƁěŲƻƫélŗÖƙ»!ơúƼŋĉĞŃǃKřĢıĴńŹŷŎƞľīČşŠ¢BÎŻűƐåƬŽbûƽĺƧ¹ũƑĂTÊĽ¥WƺƾƄ£ĚƮŅæ°ģ0wÁøōþÃM|ƤRƜ&ƕƛƋŘçƎƅįIËrÓOaćĳƱõĔíVyZEÜƘdǁ9³ťƲgƴŊūƭŦ:ĵƳòƿáÙ_pƢêìöUjºśDÒŇƚQiĨƌŚëŀĊ´Ÿč¦ŢÌäeăñāţĬ¬ƥ6qóďƖŖŧėħXƩą^żvH'

In [35]:
str([1,2,3])

'[1, 2, 3]'

In [13]:
import os
def get_file_size(file_path):
    """Return the size of the file in bytes."""
    try:
        file_size = os.path.getsize(file_path)
        return file_size
    except OSError as e:
        print(f"Error: {e}")
        return None
    
get_file_size("test.txt")

3543

In [18]:
"".join([str(i) for i in list(msg)])

'204725810331553121789163172909677472522242491171871031681387621208152114166124592261957941454411581239192562523198710047140957824817281671401252051202701761151621818576153193321023713420761211362043115611973525114537221525422820516427154131301451031396920742062031091122010818871108732431881931327415369666619423817815269124912325021687172241861682067222116783211411504789461471768015522582007439245247140239182186156381412155991532531912206842571382456322312462692546624321716223711680901401222451802259979465232176223162128916642081452431045723816922160192174652296622286107168188195231111973912518521519411422319856751159134153131249312183178812097010616713917919713018414421110011912535482083201202069622312022224524319595292151371191186224753168962242081051907111215210953197103186220106512441292132106315515795730101831695947621743614813338172692261635320024520216724224822218015217973179371272172392351883516029139128120166168109105243178169210867174362392089363203137188123312650118012324538

In [19]:
import sys

# Your string
my_string = "".join([str(i) for i in list(msg)])

# Encode the string to bytes
encoded_string = my_string.encode('utf-8')

# Get the size of the encoded string in bytes
string_data_size = len(encoded_string)

print(f"The raw size of the string data is {string_data_size} bytes")

The raw size of the string data is 2566 bytes


In [34]:
"Hello, world!".bit_length()

AttributeError: 'str' object has no attribute 'bit_length'